In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from keras import backend
from tensorflow import keras
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, MaxPool2D, Flatten, Input, BatchNormalization

2023-07-20 01:08:43.076856: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-20 01:08:43.404039: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-20 01:08:44.217025: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def load_data(train_data_dir, pred_data_dir, test_data_dir):
    
    x_train = list()
    y_train = list()
    x_pred = list()
    x_test = list()
    y_test = list()
    
    for label in os.listdir(train_data_dir):
        for img_file in os.listdir(os.path.join(train_data_dir, label)):
            img = cv2.imread(os.path.join(train_data_dir, label, img_file))
            img = cv2.resize(img, [150, 150])
            x_train.append(img)
            y_train.append(label)
            
    for label in os.listdir(test_data_dir):
        for img_file in os.listdir(os.path.join(test_data_dir, label)):
            img = cv2.imread(os.path.join(test_data_dir, label, img_file))
            img = cv2.resize(img, [150, 150])
            x_test.append(img)
            y_test.append(label)
    
    for img_file in os.listdir(pred_data_dir):
        img = cv2.imread(os.path.join(pred_data_dir, img_file))
        img = cv2.resize(img, [150, 150])
        x_pred.append(img)
        
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    x_pred = np.array(x_pred)
    x_test = np.array(x_test)
    y_test = np.array(y_test)
    
    return x_train, y_train, x_pred, x_test, y_test

In [3]:
train_data_dir = "../data/intel/seg_train/seg_train/"
test_data_dir = "../data/intel/seg_test/seg_test/"
pred_data_dir = "../data/intel/seg_pred/seg_pred/"
x_train, y_train, x_pred, x_test, y_test = load_data(train_data_dir, pred_data_dir, test_data_dir)

In [4]:
def onehot_encode(ys):
    mappings_dict = dict()
    for index, item in enumerate(list(set(ys))):
        mappings_dict[item] = index
    ys = [mappings_dict[y] for y in ys]
    ys = keras.utils.to_categorical(ys)
    return ys

In [5]:
y_train = onehot_encode(y_train)
y_test = onehot_encode(y_test)

In [6]:
x_train, y_train = shuffle(x_train, y_train)
x_test, y_test = shuffle(x_test, y_test)

In [7]:
x_train = x_train.astype(np.float64)
x_test = x_test.astype(np.float64)
x_train = x_train/255.
x_test = x_test/255.
# x_train = x_train.reshape((x_train.shape[0], 150, 150, 1))
# x_test = x_test.reshape((x_test.shape[0], 150, 150, 1))

In [8]:
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(14034, 150, 150, 3) (14034, 6) (3000, 150, 150, 3) (3000, 6)


In [9]:
model = tf.keras.Sequential([
            tf.keras.Input(shape = (150, 150, 3)),
            tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
            tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling2D((2, 2), strides=2),
            tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling2D((2, 2), strides=2),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(128, activation = 'relu'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(6, activation = 'softmax'),
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

2023-07-20 01:09:54.561717: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-20 01:09:55.051663: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-20 01:09:55.051777: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-20 01:09:55.076267: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-20 01:09:55.076369: I tensorflow/compile

In [10]:
model.fit(x_train, y_train, batch_size=16, validation_split=0.2, epochs=30)

Epoch 1/30


2023-07-20 01:10:36.987100: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8900
2023-07-20 01:10:40.457179: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-07-20 01:10:43.462809: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-07-20 01:10:43.562340: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x2bab6830 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-07-20 01:10:43.562402: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2023-07-20 01:10:43.675993: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-07-20 01:10:44.081504: I

702/702 [==============================] - 49s 47ms/step - loss: 0.9203 - accuracy: 0.6679 - val_loss: 0.8349 - val_accuracy: 0.6879
Epoch 2/30
702/702 [==============================] - 22s 31ms/step - loss: 0.6702 - accuracy: 0.7545 - val_loss: 0.8026 - val_accuracy: 0.6997
Epoch 3/30
702/702 [==============================] - 22s 31ms/step - loss: 0.5659 - accuracy: 0.7950 - val_loss: 0.7836 - val_accuracy: 0.7022
Epoch 4/30
702/702 [==============================] - 22s 31ms/step - loss: 0.4218 - accuracy: 0.8489 - val_loss: 0.5959 - val_accuracy: 0.7862
Epoch 5/30
702/702 [==============================] - 22s 31ms/step - loss: 0.2669 - accuracy: 0.9089 - val_loss: 0.6088 - val_accuracy: 0.8130
Epoch 6/30
702/702 [==============================] - 22s 31ms/step - loss: 0.1778 - accuracy: 0.9410 - val_loss: 0.6641 - val_accuracy: 0.7959
Epoch 7/30
702/702 [==============================] - 22s 31ms/step - loss: 0.1876 - accuracy: 0.9347 - val_loss: 1.8469 - val_accuracy: 0.5668
Epo

In [12]:
print(model.evaluate(x_test, y_test))

94/94 [==============================] - 2s 21ms/step - loss: 1.5062 - accuracy: 0.7553
[1.5061975717544556, 0.7553333044052124]


In [ ]:
|